
# Problem Statement

This project involves developing a simplified Human vs AI Chess Game using Python. The AI component must be powered by the Minimax algorithm, with optional Alpha-Beta Pruning. The game should include a basic GUI for human interaction and support all standard chess mechanics.



# Pieces Points

Each piece in chess has a point value associated with its strategic importance:
- **Queen**: 9 points  
- **Rook**: 5 points  
- **Bishop**: 3 points  
- **Knight**: 3 points  
- **Pawn**: 1 point  
- **King**: Infinite value (as capturing the king is the game's objective)



# Classes to Implement

The program should follow an object-oriented design with the following main classes:

- **ChessGame**: Manages the game loop, turns, and checks for end conditions.
- **Board**: Represents the chessboard and maintains the state of all pieces.
- **Move**: Represents a single move, including source, destination, and move type.
- **Piece (Abstract)**: Base class with shared behavior for all chess pieces.
- **King, Queen, Rook, Bishop, Knight, Pawn**: Inherit from Piece and implement specific movements.
- **Player (Abstract)**: Interface for Human and AI players.
- **HumanPlayer**: Handles user input through GUI.
- **AIPlayer**: Implements Minimax and Alpha-Beta Pruning.
- **Evaluation**: Contains heuristics used to evaluate board positions.



# Gameplay Features

The game should correctly implement the following mechanics:

- Turn-based play between human and AI.
- Legal move generation based on chess rules.
- Special moves: Castling, Promotion, and En Passant.
- Check and Checkmate detection.
- Stalemate handling.
- Move legality enforcement.



# Minimax and Alpha-Beta Pruning

The AI must use the Minimax algorithm to decide its moves. Alpha-Beta Pruning can be used to optimize the search process. The evaluation function should consider:
- Material count.
- King safety.
- Positional advantages such as control and mobility.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import copy
import sys

# Unicode characters for chess pieces
pieces = {
    'white': {
        'r': '♖', 'n': '♘', 'b': '♗', 'q': '♕', 'k': '♔', 'p': '♙',
    },
    'black': {
        'r': '♜', 'n': '♞', 'b': '♝', 'q': '♛', 'k': '♚', 'p': '♟',
    },
}

# Piece values for evaluation
PIECE_VALUES = {
    'Q': 9, 'R': 5, 'B': 3, 'N': 3, 'P': 1, 'K': 0  # King's value is handled specially
}
# Add this with the other constants at the top
PIECE_NAMES = {
    'P': 'Pawn',
    'R': 'Rook',
    'N': 'Knight',
    'B': 'Bishop',
    'Q': 'Queen',
    'K': 'King'
}
BLACK = 'black'
WHITE = 'white'

class Board:
    def __init__(self):
        self.grid = [[None]*8 for _ in range(8)]
        self.en_passant_target = None  # Square where en passant capture is possible
        self.castling_rights = {
            WHITE: {'kingside': True, 'queenside': True},
            BLACK: {'kingside': True, 'queenside': True}
        }
        self.setup_pieces()

    def setup_pieces(self):
        # Set up pawns
        for i in range(8):
            self.grid[1][i] = Pawn(BLACK)
            self.grid[6][i] = Pawn(WHITE)
        
        # Set up other pieces
        for i, piece in enumerate([Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook]):
            self.grid[0][i] = piece(BLACK)
            self.grid[7][i] = piece(WHITE)

    def move_piece(self, move):
        sr, sc = move.start_pos  # Start row and column
        er, ec = move.end_pos    # End row and column
    
        # Handle castling
        if isinstance(self.grid[sr][sc], King) and abs(sc - ec) == 2:
            # Kingside castling
            if ec == 6:
                self.grid[er][5] = self.grid[er][7]  # Rook h1/h8 → f1/f8
                self.grid[er][7] = None
            # Queenside castling
            elif ec == 2:
                self.grid[er][3] = self.grid[er][0]  # Rook a1/a8 → d1/d8
                self.grid[er][0] = None
    
        # Get the piece at the destination (captured piece)
        captured_piece = self.grid[er][ec]
    
        # Handle en passant capture
        if (isinstance(self.grid[sr][sc], Pawn) and 
            abs(sc - ec) == 1 and 
            captured_piece is None and 
            (er, ec) == self.en_passant_target):
            captured_piece = self.grid[sr][ec]  # Capture the pawn behind
            self.grid[sr][ec] = None
    
        # Handle pawn promotion
        if isinstance(self.grid[sr][sc], Pawn) and (er == 0 or er == 7):
            self.grid[sr][sc] = Queen(self.grid[sr][sc].color)  # Auto-queen promotion
    
        # Update castling rights and has_moved flag
        if isinstance(self.grid[sr][sc], King):
            self.castling_rights[self.grid[sr][sc].color] = {'kingside': False, 'queenside': False}
            self.grid[sr][sc].has_moved = True
        elif isinstance(self.grid[sr][sc], Rook):
            if sc == 0:  # Queenside rook
                self.castling_rights[self.grid[sr][sc].color]['queenside'] = False
            elif sc == 7:  # Kingside rook
                self.castling_rights[self.grid[sr][sc].color]['kingside'] = False
            self.grid[sr][sc].has_moved = True
    
        # Move the piece
        self.grid[er][ec] = self.grid[sr][sc]
        self.grid[sr][sc] = None
    
        # Set en passant target if pawn moves two squares
        if isinstance(self.grid[er][ec], Pawn) and abs(sr - er) == 2:
            self.en_passant_target = (sr + (er - sr) // 2, sc)
        else:
            self.en_passant_target = None
    
        return captured_piece

    def get_piece(self, row, col):
        if 0 <= row < 8 and 0 <= col < 8:
            return self.grid[row][col]
        return None

    def display(self):
        self.draw_chessboard()
       
    def draw_chessboard(self):
        fig, ax = plt.subplots(figsize=(8, 8))
        colors = ['#D18F5C', '#A66D4D']  # Light and dark brown colors
        
        for row in range(8):
            for col in range(8):
                color = colors[(row + col) % 2]
                rect = patches.Rectangle((col, 7 - row), 1, 1, linewidth=1, edgecolor='black', facecolor=color)
                ax.add_patch(rect)
        
                piece = self.grid[row][col]
                if piece:
                    color_key = 'white' if piece.color == WHITE else 'black'
                    ax.text(col + 0.5, 7 - row + 0.5, pieces[color_key][piece.symbol.lower()], 
                            fontsize=50, ha='center', va='center')
        
        # Add labels
        for i in range(8):
            ax.text(i + 0.5, -0.3, chr(ord('a') + i), ha='center', va='center', fontsize=14)
            ax.text(-0.3, i + 0.5, str(8 - i), ha='center', va='center', fontsize=14)
        
        ax.set_xlim(0, 8)
        ax.set_ylim(0, 8)
        ax.axis('off')
        plt.gca().set_aspect('equal')
        plt.title("Chess Board", fontsize=18)
        plt.show()

    
    def is_square_under_attack(self, row, col, color):
        """Check if a square is attacked by any opponent pieces without recursion"""
        for r in range(8):
            for c in range(8):
                piece = self.get_piece(r, c)
                if piece and piece.color != color:
                    # Handle each piece type separately to avoid recursion
                    if isinstance(piece, Pawn):
                        # Pawns attack diagonally
                        attack_dir = -1 if piece.color == WHITE else 1
                        if (r + attack_dir == row and 
                            abs(c - col) == 1):
                            return True
                    elif isinstance(piece, Knight):
                        # Knight moves in L-shape
                        if (abs(r - row) == 2 and abs(c - col) == 1) or \
                           (abs(r - row) == 1 and abs(c - col) == 2):
                            return True
                    else:
                        # For sliding pieces (Rook, Bishop, Queen, King)
                        dr, dc = row - r, col - c
                        if dr == 0 and dc == 0:
                            continue
                            
                        # Check if piece can attack in this direction
                        if isinstance(piece, Rook) and (dr == 0 or dc == 0):
                            step_r = dr//max(1, abs(dr)) if dr != 0 else 0
                            step_c = dc//max(1, abs(dc)) if dc != 0 else 0
                        elif isinstance(piece, Bishop) and (abs(dr) == abs(dc)):
                            step_r = dr//abs(dr) if dr != 0 else 0
                            step_c = dc//abs(dc) if dc != 0 else 0
                        elif isinstance(piece, Queen) and \
                             ((dr == 0 or dc == 0) or (abs(dr) == abs(dc))):
                            step_r = dr//max(1, abs(dr)) if dr != 0 else 0
                            step_c = dc//max(1, abs(dc)) if dc != 0 else 0
                        elif isinstance(piece, King) and \
                             abs(dr) <= 1 and abs(dc) <= 1:
                            return True
                        else:
                            continue
                            
                        # Check path is clear
                        current_r, current_c = r + step_r, c + step_c
                        while (current_r != row or current_c != col):
                            if self.get_piece(current_r, current_c):
                                break
                            current_r += step_r
                            current_c += step_c
                        else:
                            return True
        return False


class Piece:
    def __init__(self, color):
        self.color = color
    @property
    def name(self):
        return PIECE_NAMES.get(self.symbol, 'Piece')

    def get_valid_moves(self, board, row, col):
        raise NotImplementedError


class King(Piece):
    def __init__(self, color):
        super().__init__(color)
        self.symbol = 'K'
        self.has_moved = False

    def get_valid_moves(self, board, row, col):
        moves = []
        # Regular king moves (don't check for check here)
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                r, c = row + dr, col + dc
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board.get_piece(r, c)
                    if target is None or target.color != self.color:
                        moves.append((r, c))
        
        # Castling (simplified - actual check validation happens in ChessGame)
        if not self.has_moved:
            # Kingside
            if (board.castling_rights[self.color]['kingside'] and
                board.get_piece(row, 5) is None and
                board.get_piece(row, 6) is None and
                isinstance(board.get_piece(row, 7), Rook) and
                not board.get_piece(row, 7).has_moved):
                moves.append((row, 6))
                
            # Queenside
            if (board.castling_rights[self.color]['queenside'] and
                board.get_piece(row, 1) is None and
                board.get_piece(row, 2) is None and
                board.get_piece(row, 3) is None and
                isinstance(board.get_piece(row, 0), Rook) and
                not board.get_piece(row, 0).has_moved):
                moves.append((row, 2))
                
        return moves

class Queen(Piece):
    def __init__(self, color):
        super().__init__(color)
        self.symbol = 'Q'

    def get_valid_moves(self, board, row, col):
        moves = []
        # Rook-like moves
        for d in [(1,0), (-1,0), (0,1), (0,-1)]:
            for i in range(1, 8):
                r, c = row + d[0]*i, col + d[1]*i
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board.get_piece(r, c)
                    if target is None:
                        moves.append((r, c))
                    elif target.color != self.color:
                        moves.append((r, c))
                        break
                    else:
                        break
        
        # Bishop-like moves
        for d in [(1,1), (1,-1), (-1,1), (-1,-1)]:
            for i in range(1, 8):
                r, c = row + d[0]*i, col + d[1]*i
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board.get_piece(r, c)
                    if target is None:
                        moves.append((r, c))
                    elif target.color != self.color:
                        moves.append((r, c))
                        break
                    else:
                        break
        return moves


class Rook(Piece):
    def __init__(self, color):
        super().__init__(color)
        self.symbol = 'R'
        self.has_moved = False

    def get_valid_moves(self, board, row, col):
        moves = []
        for d in [(1,0), (-1,0), (0,1), (0,-1)]:
            for i in range(1, 8):
                r, c = row + d[0]*i, col + d[1]*i
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board.get_piece(r, c)
                    if target is None:
                        moves.append((r, c))
                    elif target.color != self.color:
                        moves.append((r, c))
                        break
                    else:
                        break
        return moves


class Bishop(Piece):
    def __init__(self, color):
        super().__init__(color)
        self.symbol = 'B'

    def get_valid_moves(self, board, row, col):
        moves = []
        for d in [(1,1), (1,-1), (-1,1), (-1,-1)]:
            for i in range(1, 8):
                r, c = row + d[0]*i, col + d[1]*i
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board.get_piece(r, c)
                    if target is None:
                        moves.append((r, c))
                    elif target.color != self.color:
                        moves.append((r, c))
                        break
                    else:
                        break
        return moves


class Knight(Piece):
    def __init__(self, color):
        super().__init__(color)
        self.symbol = 'N'

    def get_valid_moves(self, board, row, col):
        moves = []
        directions = [(2, 1), (1, 2), (-1, 2), (-2, 1), (-2, -1), (-1, -2), (1, -2), (2, -1)]
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                target = board.get_piece(r, c)
                if target is None or target.color != self.color:
                    moves.append((r, c))
        return moves


class Pawn(Piece):
    def __init__(self, color):
        super().__init__(color)
        self.symbol = 'P'

    def get_valid_moves(self, board, row, col):
        moves = []
        direction = -1 if self.color == WHITE else 1  # White moves up (decreasing row), black moves down (increasing row)
        start_row = 6 if self.color == WHITE else 1  # Starting row for pawns (0-based)
        
        # Forward moves
        if 0 <= row + direction < 8 and board.get_piece(row + direction, col) is None:
            moves.append((row + direction, col))
            # Initial two-square move
            if row == start_row and board.get_piece(row + 2 * direction, col) is None:
                moves.append((row + 2 * direction, col))
        
        # Capture moves (including en passant)
        for dc in [-1, 1]:
            c = col + dc
            if 0 <= c < 8:
                # Normal capture
                r = row + direction
                if 0 <= r < 8:
                    target = board.get_piece(r, c)
                    if target and target.color != self.color:
                        moves.append((r, c))
                
                # En passant capture
                if (board.en_passant_target and
                    (row + direction, c) == board.en_passant_target):
                    moves.append((row + direction, c))
        return moves



class Move:
    def __init__(self, start_pos, end_pos, piece, captured=None, move_type='normal'):
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.piece = piece
        self.captured = captured
        self.move_type = move_type  # 'normal', 'castle', 'en_passant', 'promotion'
        
    def __str__(self):
        if self.move_type == 'castle':
            side = "kingside" if self.end_pos[1] > self.start_pos[1] else "queenside"
            return f"{side} castling"
        elif self.move_type == 'en_passant':
            return f"en passant capture"
        elif self.move_type == 'promotion':
            return f"pawn promotion to {self.promotion_piece}"
        else:
            capture_msg = f" (capturing {self.captured.name})" if self.captured else ""
            return f"{self.piece.name} from {self.start_pos} to {self.end_pos}{capture_msg}"

class Player:
    def __init__(self, color):
        self.color = color

    def get_move(self, board):
        raise NotImplementedError


class HumanPlayer(Player):
    def get_move(self, board):
        while True:
            try:
                # Display the board first
                board.display()
                
                print(f"\n{self.color.capitalize()}'s turn (Human)")
                piece_input = input("Select a piece to move (e.g. e2): ").strip().lower()
                
                if len(piece_input) == 2:
                    col = ord(piece_input[0]) - ord('a')
                    row = 8 - int(piece_input[1])
                    
                    if 0 <= row < 8 and 0 <= col < 8:
                        piece = board.get_piece(row, col)
                        if piece and piece.color == self.color:
                            valid_moves = piece.get_valid_moves(board, row, col)
                            if valid_moves:
                                print(f"Valid moves for {piece.symbol} at {piece_input}: ", end='')
                                for move in valid_moves:
                                    move_str = chr(move[1] + ord('a')) + str(8 - move[0])
                                    print(move_str, end=' ')
                                print()
                                
                                dest_input = input("Enter destination (e.g. e4): ").strip().lower()
                                if len(dest_input) == 2:
                                    dest_col = ord(dest_input[0]) - ord('a')
                                    dest_row = 8 - int(dest_input[1])
                                    if (dest_row, dest_col) in valid_moves:
                                        return piece_input + dest_input
                                print("Invalid destination. Try again.")
                            else:
                                print("No valid moves for this piece. Try another.")
                        else:
                            print("Invalid piece selection. Try again.")
                    else:
                        print("Invalid coordinates. Try again.")
                else:
                    print("Invalid input format. Use format like 'e2'.")
            except ValueError:
                print("Invalid input. Please enter coordinates like 'e2'.")


class AIPlayer(Player):
    def get_move(self, board):
        print(f"\n{self.color.capitalize()}'s turn (AI)")
        # Initialize alpha and beta for the root call
        _, best_move = self.minimax(board, 3, float('-inf'), float('inf'), True)
        if best_move:
            start_col = chr(best_move.start_pos[1] + ord('a'))
            start_row = str(8 - best_move.start_pos[0])
            end_col = chr(best_move.end_pos[1] + ord('a'))
            end_row = str(8 - best_move.end_pos[0])
            return f"{start_col}{start_row}{end_col}{end_row}"
        return None

    def minimax(self, board, depth, alpha, beta, maximizing_player):
        if depth == 0:
            return Evaluation.evaluate(board, self.color), None

        valid_moves = []
        for r in range(8):
            for c in range(8):
                piece = board.get_piece(r, c)
                if piece and piece.color == (self.color if maximizing_player else (BLACK if self.color == WHITE else WHITE)):
                    for move_pos in piece.get_valid_moves(board, r, c):
                        valid_moves.append(Move((r, c), move_pos, piece))

        if not valid_moves:
            # Checkmate or stalemate
            score = Evaluation.evaluate(board, self.color)
            return score, None

        best_move = None
        
        if maximizing_player:
            max_eval = float('-inf')
            for move in valid_moves:
                new_board = copy.deepcopy(board)
                new_board.move_piece(move)
                evaluation, _ = self.minimax(new_board, depth-1, alpha, beta, False)
                
                if evaluation > max_eval:
                    max_eval = evaluation
                    best_move = move
                
                alpha = max(alpha, evaluation)
                if beta <= alpha:
                    break  # Beta cutoff
                    
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in valid_moves:
                new_board = copy.deepcopy(board)
                new_board.move_piece(move)
                evaluation, _ = self.minimax(new_board, depth-1, alpha, beta, True)
                
                if evaluation < min_eval:
                    min_eval = evaluation
                    best_move = move
                
                beta = min(beta, evaluation)
                if beta <= alpha:
                    break  # Alpha cutoff
                    
            return min_eval, best_move

class Evaluation:
    @staticmethod
    def evaluate(board, color):
        score = 0
        
        # Material score
        for row in range(8):
            for col in range(8):
                piece = board.get_piece(row, col)
                if piece:
                    value = PIECE_VALUES.get(piece.symbol.upper(), 0)
                    # King safety bonus
                    if piece.symbol.upper() == 'K':
                        value += 100  # Arbitrarily large value for king safety
                    
                    if piece.color == color:
                        score += value
                    else:
                        score -= value
        
        # Positional bonuses
        for row in range(8):
            for col in range(8):
                piece = board.get_piece(row, col)
                if piece:
                    # Center control bonus
                    if (3 <= row <= 4) and (3 <= col <= 4):
                        if piece.color == color:
                            score += 0.1
                        else:
                            score -= 0.1
        
        return score


class ChessGame:
    def __init__(self):
        self.board = Board()
        self.players = {WHITE: HumanPlayer(WHITE), BLACK: AIPlayer(BLACK)}
        self.turn = WHITE

    def play(self):
        while True:
            player = self.players[self.turn]
            
            # Get move input
            move_input = player.get_move(self.board)
            
            # For AI, we need to parse the move
            if isinstance(player, AIPlayer) and move_input:
                print(f"AI plays: {move_input}")
            
            move = self.parse_move(move_input)
            
            # Validate and apply the move
            if move and self.is_valid_move(move):
                captured_piece = self.board.move_piece(move)
                
                # Show capture message if a piece was captured
                # In ChessGame.play()
                if captured_piece:
                    print(f"{move.piece.name} ({self.turn}) captured {captured_piece.name} at {move_input[2:4]}!")
                                
                # Check for game end conditions
                if self.check_for_end_conditions():
                    self.board.display()
                    break
    
                # Switch turns
                self.turn = BLACK if self.turn == WHITE else WHITE
            else:
                if isinstance(player, HumanPlayer):
                    print("Invalid move. Please try again.")

    def parse_move(self, move_input):
        if not move_input or len(move_input) != 4:
            return None

        try:
            start_col = ord(move_input[0]) - ord('a')
            start_row = 8 - int(move_input[1])
            end_col = ord(move_input[2]) - ord('a')
            end_row = 8 - int(move_input[3])

            sr, sc, er, ec = start_row, start_col, end_row, end_col
            piece = self.board.get_piece(sr, sc)
            
            if piece and piece.color == self.turn:
                return Move((sr, sc), (er, ec), piece)
        except:
            return None
        
        return None


    
    def is_valid_move(self, move):
        piece = self.board.get_piece(*move.start_pos)
        if not piece or piece.color != self.turn:
            return False
    
        # Check if destination is in piece's valid moves
        valid_moves = piece.get_valid_moves(self.board, *move.start_pos)
        if move.end_pos not in valid_moves:
            return False
    
        # Special case: for castling, we need additional checks
        if isinstance(piece, King) and abs(move.start_pos[1] - move.end_pos[1]) == 2:
            return self.is_valid_castling(move)
    
        # Make a temporary move
        temp_board = copy.deepcopy(self.board)
        captured_piece = temp_board.move_piece(move)
        move.captured = captured_piece  # Store captured piece for later use
    
        # Check if king is in check after move
        king_pos = None
        for r in range(8):
            for c in range(8):
                p = temp_board.get_piece(r, c)
                if isinstance(p, King) and p.color == self.turn:
                    king_pos = (r, c)
                    break
            if king_pos:
                break
    
        if not king_pos:
            return False
    
        # Check if king's position is under attack
        return not temp_board.is_square_under_attack(*king_pos, self.turn)

    def check_for_end_conditions(self):
        current_color = self.turn
        king_pos = None
    
        # Find the king's position
        for r in range(8):
            for c in range(8):
                piece = self.board.get_piece(r, c)
                if piece and isinstance(piece, King) and piece.color == current_color:
                    king_pos = (r, c)
                    break
            if king_pos:
                break

        if not king_pos:
            return True  # Shouldn't happen in normal chess

        # Check if king is in check
        def is_in_check(color):
            enemy_color = BLACK if color == WHITE else WHITE
            king_r, king_c = king_pos
            for r in range(8):
                for c in range(8):
                    piece = self.board.get_piece(r, c)
                    if piece and piece.color == enemy_color:
                        if (king_r, king_c) in piece.get_valid_moves(self.board, r, c):
                            return True
            return False
    
        # Check if any legal moves exist
        legal_moves_exist = False
        for r in range(8):
            for c in range(8):
                piece = self.board.get_piece(r, c)
                if piece and piece.color == current_color:
                    valid_moves = piece.get_valid_moves(self.board, r, c)
                    for move_end in valid_moves:
                        temp_board = copy.deepcopy(self.board)
                        move = Move((r, c), move_end, piece)
                        temp_board.move_piece(move)
                        
                        # Check if this move gets out of check
                        test_game = ChessGame()
                        test_game.board = temp_board
                        test_game.turn = current_color
                        if not test_game.is_king_in_check(current_color):
                            legal_moves_exist = True
                            break
                    if legal_moves_exist:
                        break
            if legal_moves_exist:
                break
    
        if not legal_moves_exist:
            if is_in_check(current_color):
                print(f"Checkmate! {current_color.capitalize()} loses.")
            else:
                print("Stalemate! It's a draw.")
            return True
    
        return False

    def is_king_in_check(self, color):
        king_pos = None
        for r in range(8):
            for c in range(8):
                piece = self.board.get_piece(r, c)
                if piece and isinstance(piece, King) and piece.color == color:
                    king_pos = (r, c)
                    break
            if king_pos:
                break

        if not king_pos:
            return False

        enemy_color = BLACK if color == WHITE else WHITE
        king_r, king_c = king_pos
        for r in range(8):
            for c in range(8):
                piece = self.board.get_piece(r, c)
                if piece and piece.color == enemy_color:
                    if (king_r, king_c) in piece.get_valid_moves(self.board, r, c):
                        return True
        return False


if __name__ == '__main__':
    game = ChessGame()
    game.play()